In [8]:
import numpy as np

def label():
    A = np.array([['ENG','SWE','NOR','GER','DEN'],
                  ['red','green','white','yellow','blue'],
                  ['coffee','milk','beer','water','tea'],
                  ['LEXUS','BWM','BENZ','GM','FORD'],
                  ['dog','bird','cat','horse','fish']])
    return A

def encoder(A):
    B = np.arange(5*5).reshape(5,5)
    #Dict Mapping
    categories = {}
    for i in range(5):
        for j in range(5):
            categories[A[i,j]] = B[i,j]
    return categories

A = label()
data = encoder(A)
data

{np.str_('ENG'): np.int64(0),
 np.str_('SWE'): np.int64(1),
 np.str_('NOR'): np.int64(2),
 np.str_('GER'): np.int64(3),
 np.str_('DEN'): np.int64(4),
 np.str_('red'): np.int64(5),
 np.str_('green'): np.int64(6),
 np.str_('white'): np.int64(7),
 np.str_('yellow'): np.int64(8),
 np.str_('blue'): np.int64(9),
 np.str_('coffee'): np.int64(10),
 np.str_('milk'): np.int64(11),
 np.str_('beer'): np.int64(12),
 np.str_('water'): np.int64(13),
 np.str_('tea'): np.int64(14),
 np.str_('LEXUS'): np.int64(15),
 np.str_('BWM'): np.int64(16),
 np.str_('BENZ'): np.int64(17),
 np.str_('GM'): np.int64(18),
 np.str_('FORD'): np.int64(19),
 np.str_('dog'): np.int64(20),
 np.str_('bird'): np.int64(21),
 np.str_('cat'): np.int64(22),
 np.str_('horse'): np.int64(23),
 np.str_('fish'): np.int64(24)}

In [14]:
from ortools.sat.python import cp_model

# Define categories
categories = data

# Define the model
model = cp_model.CpModel()

num_rows = 5
num_cols = 5

X = [[model.NewIntVar(0, 24, f'X_{i}_{j}') for j in range(num_cols)] for i in range(num_rows)]

a = model.NewIntVar(0, 4, 'a')
b = model.NewIntVar(0, 4, 'b')
c = model.NewIntVar(0, 3, 'c')  # c can't be 4 because of c+1 constraint
d = model.NewIntVar(0, 4, 'd')
e = model.NewIntVar(0, 3, 'e')  # e can't be 4 because of e+1 constraint
f = model.NewIntVar(1, 4, 'f')  # f can't be 0 because of f-1 constraint
g = model.NewIntVar(0, 4, 'g')
h = model.NewIntVar(0, 4, 'h')
i = model.NewIntVar(0, 4, 'i')

model.Add(X[0][a] == categories['Eng']) 
model.Add(X[1][a] == categories['red'])

model.Add(X[0][b] == categories['SWE'])
model.Add(X[4][b] == categories['dog'])

model.Add(X[1][c] == categories['green'])
model.Add(X[1][c+1] == categories['white'])
model.Add(c != 4)

#model.Add(X[1][c] == categories['green'])
model.Add(X[2][c] == categories['coffee'])

model.Add(X[3][d] == categories['LEXUS'])
model.Add(X[4][d] == categories['bird'])

model.Add(X[1][e] == categories['yellow'])
model.Add(X[3][e] == categories['BWM'])

model.Add(X[2][2] == categories['milk'] )

model.Add(X[0][0] == categories['NOR'])

model.Add(X[3][f] == categories['BENZ'])
model.Add(X[4][f-1] == categories['cat'])
model.Add(f != 0)

#model.Add(X[3][e] == categories['BMW'])
model.Add(X[4][e+1] == categories['horse'])
model.Add(e != 5)

model.Add(X[3][g] == categories['GM'])
model.Add(X[2][g] == categories['beer'])

model.Add(X[0][h] == categories['GER'])
model.Add(X[3][h] == categories['FORD'])

model.Add(X[1][1] == categories['NOR'])

model.Add(X[3][f] == categories['BENZ'])
model.Add(X[2][f+1] == categories['water'])
model.Add(f != 4)
#model.Add(X[2][f-1] == catagories['water'])
#model.Add(f != 0)

model.Add(X[0][i] == categories['DEN'])
model.Add(X[2][i] == categories['tea'])

# Create the solver and solve the model
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('Solution:')
    for r in range(num_rows):
        for c in range(num_cols):
            print(f'X[{r}][{c}] = {solver.Value(X[r][c])}')
else:
    print('No solution found')

TypeError: list indices must be integers or slices, not IntVar

In [16]:
from ortools.sat.python import cp_model

# Define categories
categories = {
    'Eng': 1, 'red': 2, 'SWE': 3, 'dog': 4, 'green': 5, 'white': 6,
    'coffee': 7, 'LEXUS': 8, 'bird': 9, 'yellow': 10, 'BMW': 11,
    'milk': 12, 'NOR': 13, 'BENZ': 14, 'cat': 15, 'horse': 16,
    'GM': 17, 'beer': 18, 'GER': 19, 'FORD': 20, 'DEN': 21, 'tea': 22,
    'water': 23
}

# Define the model
model = cp_model.CpModel()

num_rows = 5
num_cols = 5
num_categories = len(categories)

# Create variables
X = [[[model.NewBoolVar(f'X_{i}_{j}_{k}') for k in range(num_categories)] for j in range(num_cols)] for i in range(num_rows)]

# Ensure each cell contains exactly one category
for i in range(num_rows):
    for j in range(num_cols):
        model.Add(sum(X[i][j]) == 1)

# Ensure each category appears exactly once in the grid
for k in range(num_categories):
    model.Add(sum(X[i][j][k] for i in range(num_rows) for j in range(num_cols)) == 1)

# Helper function to convert category names to indices
def cat_index(category):
    return categories[category] - 1

# Add constraints for the given problem
model.Add(X[0][0][cat_index('NOR')] == 1)
model.Add(X[0][1][cat_index('Eng')] == 1)
model.Add(X[1][1][cat_index('red')] == 1)
model.Add(X[0][2][cat_index('SWE')] == 1)
model.Add(X[4][2][cat_index('dog')] == 1)
model.Add(X[1][3][cat_index('green')] == 1)
model.Add(X[1][4][cat_index('white')] == 1)
model.Add(X[2][3][cat_index('coffee')] == 1)
model.Add(X[3][4][cat_index('LEXUS')] == 1)
model.Add(X[4][4][cat_index('bird')] == 1)
model.Add(X[1][0][cat_index('yellow')] == 1)
model.Add(X[3][0][cat_index('BMW')] == 1)
model.Add(X[2][2][cat_index('milk')] == 1)
model.Add(X[3][1][cat_index('BENZ')] == 1)
model.Add(X[4][0][cat_index('cat')] == 1)
model.Add(X[4][1][cat_index('horse')] == 1)
model.Add(X[3][3][cat_index('GM')] == 1)
model.Add(X[2][3][cat_index('beer')] == 1)
model.Add(X[0][3][cat_index('GER')] == 1)
model.Add(X[3][2][cat_index('FORD')] == 1)
model.Add(X[2][4][cat_index('water')] == 1)
model.Add(X[0][4][cat_index('DEN')] == 1)
model.Add(X[2][0][cat_index('tea')] == 1)

# Create the solver and solve the model
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('Solution:')
    for i in range(num_rows):
        for j in range(num_cols):
            for k in range(num_categories):
                if solver.Value(X[i][j][k]):
                    print(f'X[{i}][{j}] = {list(categories.keys())[k]}')
else:
    print('No solution found')


No solution found
